<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-и-предобработка-данных" data-toc-modified-id="Изучение-и-предобработка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение и предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.


- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет

## Изучение и предобработка данных
### Изучение данных

In [1]:
# Загрузим все необходимые библиотеки.

import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
#Изучим данные

data = pd.read_csv('/datasets/insurance.csv')

In [3]:
display (data.head())
data.info()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
# Изучим наличие дубликатов

data.duplicated().value_counts()

False    4847
True      153
dtype: int64

In [5]:
# Исследуем данные на наличие пропусков

data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

По результатам исследования данных имеем следующие выводы:
1. Целевым категориальным признаком является столбец "Страховые выплаты";
1. Столбцы "возраст" и "зарплата" необходимо привести к правильному формату;
1. Необходимо избавиться от явных дубликатов.
1. Пропусков в данных не обнаружено.

### Предобработка данных

In [6]:
# Выделим features и target в отдельные переменные:
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

# Приведём данные к правильному формату:
data['Зарплата'] = data['Зарплата'].astype(int)
data['Возраст'] = data['Возраст'].astype(int)

# Избавимя от дубликатов:
data.drop_duplicates(inplace=True)

# Рассмотрим данные после предобработки
display(data.head())
print('Сумма явных дубликатов:', data.duplicated().sum())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


Сумма явных дубликатов: 0


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу?**

**Ответ:** Нет.

**Обоснование:** обозначим исходную матрицу признаков $w_1$, а умноженную на другую матрицу $w_2$, тогда:


$$
w_1 = (X^T X)^{-1} X^T y
$$

$$
w_2 = ((XP)^T (XP))^{-1} (XP)^T y
$$

Раскроем скобки:

$$
w_2 = (P^T X^T X P)^{-1} P^{T}X^{T}y
$$

$$
w_2 = ((X^TX)P)^{-1} P^{T^{-1}} P^{T}X^{T}y
$$

$$
w_2 = P^{-1}(X^TX)^{-1} P^{T^{-1}} P^{T}X^{T}y
$$

$P^{T^{-1}}P^{T}$ являtтся единичной матрицей ($E$). Умножение матрицы на единичную эквивалентно умножению числа на 1. Таким образом:

$$
w_2 =P^{-1}*[(X^{T}X)^{-1}X^{T}y]
$$

$$
w_2 = P^{-1}w_1
$$

**ЧТД.**

## Алгоритм преобразования

В предыдущем пункте было теоретически доказано, что умножение матрицы на обратимую не должно повлять на качество линейной регрессии. С целью эмперического подтверждения того же тезиса составим алгоритм преобразования матрицы признаков с поэтапным измерением метрики $R^2$:

1. Присвоить переменной модель линейной регрессии;
1. Обучить модель на исходной матрице признаков $X$;
1. Сделать прогноз на основе исходных данных;
1. Вычислить метрику $R_1^2$;
1. Умножить исходную матрицу $X$ на обратимую матирцу $P$;
1. Обучить модель на преобразованной матрице;
1. Сделать прогноз на преобразованных данных;
1. Вычислить метрику $R_2^2$;
1. Сравнить полученные коэффициенты детерминации.

Таким образом будет получен однозначный вывод о возможном изменении матрицы признаков при умножении на обратимую.

## Проверка алгоритма

In [7]:
# 1. Присвоим переменной `model` модель линейной регрессии
model = LinearRegression()

# 2. Обучим модель на исходной матрице признаков, представленной переменными features и target:
model.fit(features, target)

# 3. Сделаем прогноз на основе исходных данных:
predictions1 = model.predict(features)

# 4. Вычислим метрику R2 для исходной выборки:
r2_old = r2_score(target, predictions1)
print("R2 исходного набора данных:", r2_old)

R2 исходного набора данных: 0.4249455028666801


In [8]:
# 5. Чтобы умножить исходную матрицу на обратимую создадим матрицу `random` той же размерности и проверим её на обратимость:
random = np.random.randn(4, 4)
print('Сгенерированная матрица:\n', random)

try:
    np.linalg.inv(random)
    print('\nМатрица обратима')
except:
    print('\nМатрица необратима')

Сгенерированная матрица:
 [[-0.12479937 -0.11838017  0.1824721   1.08985344]
 [-1.19632646 -0.54606649 -0.4291928   0.12286849]
 [ 0.93046785  0.66666656 -1.64165867  0.7597778 ]
 [ 1.78376329  1.10078856  0.27784827  1.35215216]]

Матрица обратима


In [9]:
# Проведём непосредственное умножение `random` на `features` и выведем на экран:
features_new = features @ random
display('Матрица признаков после преобразования:', features_new)

'Матрица признаков после преобразования:'

,0,1,2,3
0,46103.814771,33045.255134,-81443.406790,37692.458294
1,35304.530917,25309.311068,-62402.494613,28878.560350
2,19505.131312,13984.161864,-34487.278735,15958.896902
3,38778.953875,27790.729796,-68465.624037,31688.018635
4,24251.588856,17384.589014,-42859.126305,19834.730646
...,...,...,...,...
4995,33187.772510,23786.907962,-58618.676346,27130.211939
4996,48717.623826,34915.862351,-86037.229198,39817.886190
4997,31522.500998,22591.276683,-55660.257191,25761.628958
4998,30405.205893,21791.167084,-53690.664848,24852.583345


In [10]:
# 6. Обучим модель на преобразованной матрице:
model = LinearRegression()
model.fit(features_new, target)

# 7. Сделаем прогноз на основе преобразованных данных:
predictions2 = model.predict(features_new)

# 8. Вычислим метрику R2 для преобразованной выборки:
r2_new = r2_score(target, predictions2)
print("R2 преобразованного набора данных:", r2_new)

R2 преобразованного набора данных: 0.42494550286668986


In [11]:
# Сравним полученные коэффициенты детерминации:
print('Разность метрик R2 составила:', r2_old-r2_new)

Разность метрик R2 составила: -9.769962616701378e-15


Эмперически **доказано, что качество линейной регрессии незначительно мало искажается при умножении исходных данных на обратимую матрицу.**

В целях самопроверки восстановим матрицу путём умножения модифицированного `features` на обратную матрицу и сравним округлённые резльтаты:

In [20]:
features.columns = range(features.columns.size)
restored_data = features_new @ (np.linalg.inv(random))
np.round(restored_data) == np.round(features)

,0,1,2,3
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True
...,...,...,...,...
4995,True,True,True,True
4996,True,True,True,True
4997,True,True,True,True
4998,True,True,True,True


**Проверка пройдена успешно.**